In [ ]:
import nilearn
from nilearn import plotting
from nilearn import datasets

In [ ]:
def threshold_p(feature_list, p):
    '''
    '''
    absolute_feautres = np.abs(feature_list)
    absolute_feautres.sort()
    absolute_feautres = absolute_feautres[::-1]
    threshold_index = int(len(feature_list) * p)
    threshold = absolute_feautres[n - 1]
    return threshold

In [ ]:
def threshold_n(feature_list, n):
    absolute_feautres = np.abs(feature_list)
    absolute_feautres.sort()
    absolute_feautres = absolute_feautres[::-1]
    threshold = absolute_feautres[n - 1]
    return threshold

In [ ]:
def draw_glassbrain(importance_df):
    '''
    Draw glass-brain including the top 5 connetivity features based on importance_df.
    
    Parameters
    ----------------------------
            importance_df - DataFrame
                DataFrame containing coefficients or feature importance of features of a model
    
    '''
    # initiate atlas and labels
    with open("../data/Schaefer2018_100Parcels_7Networks_order_info.txt", 'r') as f:
        parcels_text = f.read()
    # select only even rows, which contain the labels 
    lines = parcels_text.split('\n')[:-1]
    lines = lines[::2]
    # remove prefix from label: "7Networks_"
    parcel_list = [parcel[10:] for parcel in lines]

    schaefer = nilearn.datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7).maps
    coordinates = plotting.find_parcellation_cut_coords(labels_img=schaefer)

    for i in range(len(importance_df)):
        # extract each node's name (splitted by "__")
        node1_name = importance_df.loc[i, 'Feature'].split('__')[0]
        node2_name = importance_df.loc[i, 'Feature'].split('__')[1] 
        importance_df.loc[i, 'Node1'] = int(parcel_list.index(node1_name))
        importance_df.loc[i, 'Node2'] = int(parcel_list.index(node2_name))

    # create features matrix
    node_number = 100
    features_matrix = np.zeros([node_number, node_number])
    for i in range(len(importance_df)):
        row, col = int(importance_df.loc[i, 'Node1']), int(importance_df.loc[i, 'Node2'])
        features_matrix[row, col] = importance_df.loc[i, 'Beta']
        # symmetric matrix, so [row, col] == [col, row]
        features_matrix[col, row] = importance_df.loc[i, 'Beta']
    
    # draw brain
    colors_list = ['purple', 'blue', 'green', 'violet', 'wheat', 'orange', 'red']
    net_names = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 'Cont', 'Default']
    color_dict = dict(zip(net_names, colors_list))
    node_colors = []
    for name in parcel_list:
        net = name.split('_')[1]
        node_colors.append(color_dict[net])
    # find top 5 features 
    thresh = threshold_n(importance_df.Beta.to_list(), 5)
    _ = plotting.plot_connectome(features_matrix, coordinates, edge_threshold = thresh,
                                 display_mode = 'lzry', node_size = 25, node_color = node_colors, colorbar = True)